In [ ]:
#======================== CОЗДАНИЕ БД (НЕ ОБЯЗАТЕЛЬНО)================================#
!apt install postgresql postgresql-contrib -y > /dev/null
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER PASSWORD 'root';"
!sudo -u postgres createdb -O root clearmedia
!pip install psycopg2-binary SQLAlchemy > /dev/null

In [ ]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="clearmedia",
    user="root",
    password="root",
)
conn.autocommit = True
cur = conn.cursor()
cur.execute("SELECT version();")
print(cur.fetchone())

In [ ]:
schema_sql = open("/content/drive/MyDrive/hack_2/database_schema.sql", encoding="utf-8").read()
cur.execute(schema_sql)

In [ ]:
#=================================ОСНОВНОЙ ПАЙПЛАЙН=================================#
!pip install -q torch transformers spacy
!python -m spacy download en_core_web_sm -q
!pip install -q google-trans-new

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/hack_2')
print(f"Рабочая папка: {os.getcwd()}")
print(f"Файлы: {os.listdir('.')}")

required_files = [
    'sample.csv',
    'data_loader.py',
    'text_preprocessor.py',
    'translator.py',
    'entity_extractor_ner_ensemble.py',
    'entity_deduplicator.py',
    'risk_classifier.py',
    'output_formatter.py',
    'relationship_extractor_hybrid_pro.py'
]

print("\nПРОВЕРКА ФАЙЛОВ:")
for f in required_files:
    status = "✅" if os.path.exists(f) else "❌"
    print(f"{status} {f}")

print("\nГотово к запуску пайплайна!")

In [ ]:
import json
import logging
import os
from typing import Dict, List, Any

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(message)s')
logger = logging.getLogger(__name__)

def safe_import(module_name, class_name):
    try:
        module = __import__(module_name)
        return getattr(module, class_name)()
    except Exception as e:
        print(f"{module_name}.{class_name}: {e}")
        return None

print("Загрузка модулей...")
data_loader = safe_import('data_loader', 'DataLoader')
text_preprocessor = safe_import('text_preprocessor', 'TextPreprocessor')
translator = safe_import('translator', 'Translator') or None
ner_extractor = safe_import('entity_extractor_ner_ensemble', 'NEREnsembleExtractor')
entity_deduplicator = safe_import('entity_deduplicator', 'EntityDeduplicator')
risk_classifier = safe_import('risk_classifier', 'RiskClassifier')
output_formatter = safe_import('output_formatter', 'OutputFormatter')
relation_extractor = safe_import('relationship_extractor_hybrid_pro', 'RelationExtractorHybridPro')
print("Все модули готовы")

def to_serializable(obj):
    """Конвертирует Entity объекты в dict для JSON"""
    if hasattr(obj, '__dict__'):
        return obj.__dict__
    if hasattr(obj, 'to_dict'):
        return obj.to_dict()
    if isinstance(obj, list):
        return [to_serializable(i) for i in obj]
    if isinstance(obj, dict):
        return {k: to_serializable(v) for k, v in obj.items()}
    return obj

class NewsAnalysisPipelineHybrid:
    def process_article(self, article: Dict[str, Any]) -> Dict[str, Any]:
        raw_id = article.get('id', 'unknown')
        article_id = str(raw_id)[:8]
        title = str(article.get('title', 'No title'))[:60]
        text = article.get('content', '') or article.get('text', '')
        
        print(f"[{article_id}] {title}")
        result = {'article_id': article_id, 'title': title}
        
        try:
            # 1. ПРЕДОБРАБОТКА
            cleaned_text = text
            if text_preprocessor:
                cleaned_text = text_preprocessor.preprocess(text) or text
            
            print(f"{len(cleaned_text)} символов")
            
            # 2. NER
            entities = {}
            if ner_extractor:
                ner_result = ner_extractor.extract(cleaned_text)
                if ner_result:
                    entities = ner_result.get("entities", {})
                    print(f"   👥 {len(entities.get('all', []))} сущностей")
            
            # 3. Дедупликация
            if entity_deduplicator and entities:
                entities.setdefault('persons', [])
                entities.setdefault('organizations', [])
                entities = entity_deduplicator.deduplicate_entities(entities) or entities
            
            # 4. ОТНОШЕНИЯ
            relations = []
            if relation_extractor and entities.get('all'):
                relations = relation_extractor.extract_relationships(cleaned_text, entities) or []
                print(f"   🔗 {len(relations)} отношений")
            
            # 5. Knowledge Graph
            knowledge_graph = {
                'nodes': [],
                'edges': []
            }
            
            # Nodes (поддержка Entity и dict)
            for entity in entities.get('all', []):
                if hasattr(entity, 'name'):
                    name = entity.name
                    etype = getattr(entity, 'entity_type', getattr(entity, 'type', 'UNKNOWN'))
                elif isinstance(entity, dict):
                    name = entity.get('name', str(entity))
                    etype = entity.get('type', entity.get('entity_type', 'UNKNOWN'))
                else:
                    name = str(entity)
                    etype = 'UNKNOWN'
                
                knowledge_graph['nodes'].append({
                    'id': name,
                    'type': etype,
                    'name': name
                })
            
            # Edges
            for rel in relations:
                if hasattr(rel, 'to_dict'):
                    knowledge_graph['edges'].append(rel.to_dict())
                elif isinstance(rel, dict):
                    knowledge_graph['edges'].append(rel)
            
            result.update({
                'entities': to_serializable(entities), 
                'knowledge_graph': knowledge_graph,
                'success': True
            })
            
            print(f"{len(knowledge_graph['nodes'])} nodes | {len(knowledge_graph['edges'])} edges")
            
        except Exception as e:
            result['error'] = str(e)
            print(f"{str(e)[:60]}")
        
        return result
    
    def run(self, max_articles: int = 10):
        print("\nЗАПУСК АНАЛИЗА")
        print("=" * 50)
        
        # Загрузка
        articles = []
        try:
            if data_loader:
                all_articles = data_loader.load('sample.csv')
                articles = all_articles[:max_articles]
        except:
            print("Fallback на тестовые")
            articles = [{'id': i, 'title': f'Test {i}', 'content': 'Ilham Aliyev CEO SOCAR Baku.'} for i in range(max_articles)]
        
        print(f"📄 {len(articles)} статей")
        
        results = []
        for i, article in enumerate(articles, 1):
            result = self.process_article(article)
            results.append(result)
            print(f"[{i}/{len(articles)}] {'✅' if result.get('success') else '❌'}")
        
        # JSON‑сериализация
        serializable_results = to_serializable(results)
        
        # СОХРАНЕНИЕ
        output_file = 'results_hybrid_final.json'
        print(f"\nСохранение {output_file}...")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(serializable_results, f, ensure_ascii=False, indent=2)
        print("ФАЙЛ ГОТОВ!")
        
        # СТАТИСТИКА
        successful = sum(1 for r in results if r.get('success'))
        total_relations = sum(len(r.get('knowledge_graph', {}).get('edges', [])) for r in results)
        
        print(f"\nРЕЗУЛЬТАТЫ:")
        print(f"Успешно: {successful}/{len(results)}")
        print(f"Отношений: {total_relations}")
        
        if total_relations > 0:
            print("\nТОП-5 ОТНОШЕНИЙ:")
            all_edges = []
            for r in results:
                all_edges.extend(r.get('knowledge_graph', {}).get('edges', []))
            
            for edge in sorted(all_edges, key=lambda x: x.get('confidence', 0), reverse=True)[:5]:
                print(f"   {edge.get('source_entity', '?')} → [{edge.get('relation_type', '?')}] → {edge.get('target_entity', '?')} ({edge.get('confidence', 0):.2f})")
        
        print(f"\nresults_hybrid_final.json — ГОТОВО!")
        return serializable_results

# ФИНАЛЬНЫЙ ЗАПУСК
pipeline = NewsAnalysisPipelineHybrid()
results = pipeline.run(max_articles=3)
